<a href="https://colab.research.google.com/github/agleba/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install hyperopt

In [29]:
import pandas as pd
import numpy as np
 
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

x_train, y_train = train['features'], train['labels']
x_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, x_train, y_train)
  return predict(model_trained, x_test, y_test)


In [0]:
def train_model(model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  


  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


  model.fit(
      x_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (x_train, y_train)),
      callbacks=[tensorboard_callback]

)
  
  return model

def predict(model_trained, x_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax( y_pred_prob, axis=1 )

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size =(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size =(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size =(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size =(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size =(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size =(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),


  Flatten(),
  
  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax'),
])



#train_and_predict(get_cnn_v5(input_shape, num_classes))

In [26]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, x_train, y_train)

predict(model_trained, x_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.9203 - accuracy: 0.2166 - val_loss: 1.5224 - val_accuracy: 0.5113
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.1378 - accuracy: 0.6280 - val_loss: 0.3327 - val_accuracy: 0.8921
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.5380 - accuracy: 0.8291 - val_loss: 0.1220 - val_accuracy: 0.9657
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3232 - accuracy: 0.9003 - val_loss: 0.0692 - val_accuracy: 0.9791
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2488 - accuracy: 0.9238 - val_loss: 0.0511 - val_accuracy: 0.9870


0.953514739229025

In [28]:
model_trained.evaluate(x_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1594 - accuracy: 0.9535


0.9535147547721863

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size =(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size =(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size =(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size =(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size =(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size =(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),


  Flatten(),
  
  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax'),
])


In [0]:
def func_obj(params):
  model = get_model( params )
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  

  model.fit(
      x_train, 
      y_train,
      batch_size=128, #int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
)
  
  score = model.evaluate(x_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [51]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj, 
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4005541769353813, 'dropout_cnn_block_three': 0.4951357310577966, 'dropout_cnn_block_two': 0.38802578132741555, 'dropout_dense_block_one': 0.36393007230986674, 'dropout_dense_block_two': 0.38899716314629984}
accuracy=0.9124716520309448
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.44594051105791965, 'dropout_cnn_block_three': 0.4637595640521095, 'dropout_cnn_block_two': 0.4425781197072558, 'dropout_dense_block_one': 0.6440693300016196, 'dropout_dense_block_two': 0.3311805940429217}
accuracy=0.9310657382011414
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.3654134637328182, 'dropout_cnn_block_three': 0.4894259008336036, 'dropout_cnn_block_two': 0.3013022494305681, 'dropout_dense_block_one': 0.428809259493892, 'dropout_dense_block_two': 0.49178647472313464}
accuracy=0.9451247453689575
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4822328119351956, 'dropout_cnn_block_three': 0.43856606811398846, 'dropout_cnn_block_two': 0.32927483386